<a href="https://colab.research.google.com/github/ntwsut77/Thai_Emotion_TTS/blob/main/Thai_Emotion_TTS_442.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🇹🇭 Emotion-Aware Thai News TTS

OFF Project — Demo pipeline for **Emotion-Aware Thai Text-to-Speech for News Narration** using:

- Whisper (ASR) for optional speech input → text
- PyThaiNLP for Thai text normalization & sentence segmentation
- Rule-based Emotion Classifier (Thai keywords) or manual emotion selection
- Massively Multilingual Speech (MMS): Thai Text-to-Speech
- Gradio web UI for interactive demo


## 🔁 System Diagram

```text
          ┌────────────────────┐
          │  Text Input (TH)  │
          └─────────┬─────────┘
                    │
           OR       │
                    ▼
          ┌────────────────────┐
          │  Audio Input (TH) │
          └─────────┬─────────┘
                    │  (librosa / ffmpeg)
                    ▼
          ┌────────────────────┐
          │  Whisper ASR       │  →  Thai transcript
          └─────────┬─────────┘
                    ▼
          ┌────────────────────┐
          │  PyThaiNLP         │  →  cleaned, segmented text
          └─────────┬─────────┘
                    ▼
          ┌───────────────────────────────────────┐
          │  Emotion Module                       │
          │  - Rule-based Thai emotion classifier │
          │  - or manual emotion selection        │
          └─────────┬────────────────────────────┘
                    ▼
          ┌───────────────────────────────────────┐
          │ Massively Multilingual Speech (MMS): Thai Text-to-Speech    │
          │ - Emotion selection           │
          │ - Speaking rate        │
          └─────────┬────────────────────────────┘
                    ▼
          ┌────────────────────┐
          │  Output WAV / UI   │
          └────────────────────┘
```


## 1️⃣ Install Dependencies

Run this cell once in Colab. It installs the required libraries for ASR, NLP, audio processing, facebook/mms-tts-tha, and Gradio.


In [ ]:
!pip install -q pandas scikit-learn gradio pythainlp librosa soundfile torch transformers scipy numpy

## 2️⃣ ASR + NLP + Train TF-IDF + SVM Emotion Classifier

- Whisper (via `transformers` pipeline) for Thai speech → text
- PyThaiNLP for normalization and Thai sentence tokenization
- Simple Thai keyword-based emotion classifier for demo purposes


# **Load Dataset**

In [ ]:
import pandas as pd
df = pd.read_parquet('/content/corpus.parquet')
df.head()

# Auto Label **Emotions**

In [ ]:
def label_emotion(text):
    t=str(text)
    if any(k in t for k in ['ดีใจ','ชนะ','สำเร็จ']): return 'happy'
    if any(k in t for k in ['เสียชีวิต','สูญเสีย','เศร้า']): return 'sad'
    if any(k in t for k in ['โกรธ','วิจารณ์','เดือด']): return 'angry'
    if any(k in t for k in ['ด่วน','ล่าสุด','ประกาศ']): return 'excited'
    return 'news'
df['emotion']=df['text'].apply(label_emotion)
df['emotion'].value_counts()

# **Train TF-IDF + SVM Emotion Classifier**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['emotion'], test_size=0.2, random_state=42)

tfidf = TfidfVectorizer(max_features=20000)
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

clf = LinearSVC(class_weight='balanced')
clf.fit(X_train_vec, y_train)

print('Accuracy:', clf.score(X_test_vec, y_test))

# Whisper + **Preprocessing**

In [ ]:
from transformers import pipeline
import librosa
from pythainlp.util import normalize
# เราไม่ต้อง import word_tokenize แล้ว เพราะจะไม่ใช้การตัดคำครับ

# 1. ตั้งค่า Whisper (ใส่ chunk_length_s=30 เพื่อกัน Error ไฟล์เสียงยาว)
asr = pipeline('automatic-speech-recognition',
               model='openai/whisper-large-v3',
               device=0,
               chunk_length_s=30)

def speech_to_text(path):
    # ส่ง path ให้ pipeline จัดการได้เลย (รองรับไฟล์ยาว)
    return asr(path, batch_size=8)['text']

def preprocess(txt):
    # 1. ป้องกันกรณี input เป็น None
    if not txt or not isinstance(txt, str):
        return ""

    # 2. Normalize ข้อความ (ลบตัวอักษรขยะ, สระลอย)
    txt = normalize(txt.strip())

    # 3. 🔥 ตัดส่วน word_tokenize ออกไปเลย 🔥
    # ส่งข้อความกลับไปตรงๆ เพื่อให้ TTS อ่านแบบต่อเนื่องที่สุด ไม่ต้องเว้นวรรคตามคำ
    return txt

## 3️⃣  Text-to-Speech (TTS)
Massively Multilingual Speech (MMS): Thai Text-to-Speech

### 🔐 model detail
VITS (Variational Inference with adversarial learning for end-to-end Text-to-Speech) is an end-to-end speech synthesis model that predicts a speech waveform conditional on an input text sequence. It is a conditional variational autoencoder (VAE) comprised of a posterior encoder, decoder, and conditional prior.


In [ ]:
from transformers import VitsModel, AutoTokenizer
import torch, numpy as np
import scipy.io.wavfile as wavfile

MODEL_ID = "facebook/mms-tts-tha"
tts_model = VitsModel.from_pretrained(MODEL_ID)
tts_tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)


In [ ]:
PROSODY_MAP = {
    # ข่าว: ความเร็วปกติ, เสียงนิ่งๆ (Safe Zone)
    'news':    {'speed': 1.0,  'noise': 0.5},

    # ดีใจ: เร็วขึ้นนิดหน่อย, มีลูกเล่นเสียงพอประมาณ
    'happy':   {'speed': 1.1,  'noise': 0.7},

    # เศร้า: ช้าลง (แต่อย่ายานจนน่ารำคาญ), เสียงเรียบต่ำ
    'sad':     {'speed': 0.89, 'noise': 0.3},

    # โกรธ: เร็ว กระชาก, เสียงแข็ง (Noise สูงสุดที่ยังปลอดภัย)
    'angry':   {'speed': 1.3, 'noise': 0.8},

    # ตื่นเต้น: เร็วมาก, เสียงสูง (Noise อย่าเกิน 0.85 เดี๋ยวเสียงแตก)
    'excited': {'speed': 1,  'noise': 0.85},
}

In [1]:
from scipy.io import wavfile
import torch
import numpy as np

def cloud_tts(text, emotion):
    # 1. ดึงค่า Config ตามอารมณ์ที่เลือก
    cfg = PROSODY_MAP.get(emotion, PROSODY_MAP["news"])

    # 2. 🔥 แก้ไขจุดที่ Error:
    # แทนที่จะส่งค่าเข้า forward() เราจะยัดค่าใส่ Config ของโมเดลโดยตรงแทน
    tts_model.config.speaking_rate = cfg['speed']  # ปรับความเร็ว
    tts_model.config.noise_scale = cfg['noise']    # ปรับความเหวี่ยงของเสียง

    # 3. เตรียม Input
    inputs = tts_tokenizer(text, return_tensors="pt")

    # 4. สร้างเสียง
    try:
        with torch.no_grad():
            # ลบ noise_scale/length_scale ออกจากตรงนี้ เพราะเราตั้งค่าใน config แล้ว
            output = tts_model(**inputs)

        audio = output.waveform[0].numpy()

        # 5. Normalize เสียง (ป้องกันเสียงแตก)
        max_val = np.max(np.abs(audio))
        if max_val > 0:
            audio = audio / max_val * 32767

        audio = audio.astype(np.int16)

        # 6. บันทึกไฟล์
        output_file = "out.wav"
        wavfile.write(output_file, tts_model.config.sampling_rate, audio)

        return output_file

    except Exception as e:
        print(f"❌ Error in cloud_tts: {e}")
        return None

## 4️⃣ Gradio Web App

- Input: Thai text **or** uploaded Thai speech audio
- Options:
  - Automatic emotion detection from text (rule-based)
  - Manual override of emotion
  - Speaker (voice) selection
  - Speaking rate and pitch sliders (for fine-tuning prosody)


In [ ]:
import gradio as gr

def run(text_input, audio_input, use_auto, manual_emo):
    try:
        print("--- NEW RUN ---")
        final_text = ""

        # 1. เช็คว่ามีไฟล์เสียงไหม (ให้ความสำคัญเสียงก่อน)
        if audio_input is not None:
            print("🎤 Processing Audio...")

            # ดึง Path ไฟล์แบบปลอดภัย
            if isinstance(audio_input, dict):
                audio_path = audio_input.get("name") or audio_input.get("data")
            else:
                audio_path = audio_input

            # เรียก Whisper
            final_text = speech_to_text(audio_path)
            print(f"📝 ASR Result: '{final_text}'")

        # 2. ถ้าไม่มีเสียง ให้ใช้ Text ที่พิมพ์มา
        else:
            final_text = text_input
            print(f"⌨️ Using Text Input: '{final_text}'")

        # 3. 🔥 จุดสำคัญ: เช็คว่ามีข้อความจริงไหม ถ้าไม่มีให้หยุดเลย (กัน TTS พัง)
        if not final_text or final_text.strip() == "":
            msg = "⚠️ ไม่พบเสียงพูด หรือถอดความไม่ได้ (Silence detected)"
            print(msg)
            return None, msg, "Error"

        # 4. Preprocess (ตัดคำ)
        processed_text = preprocess(final_text)
        print(f"✂️ Processed: '{processed_text}'")

        # เช็คอีกรอบเผื่อ preprocess แล้วข้อความหายหมด (เช่น มีแต่สัญลักษณ์)
        if not processed_text or processed_text.strip() == "":
             return None, "Error: ข้อความมีแต่อักขระพิเศษที่อ่านไม่ได้", "Error"

        # 5. ทำนายอารมณ์
        if use_auto:
            emo = clf.predict(tfidf.transform([processed_text]))[0]
        else:
            emo = manual_emo
        print(f"😊 Emotion: {emo}")

        # 6. สร้างเสียง (TTS)
        # ถ้าพังตรงนี้ จะไม่ทำให้เว็บค้าง แต่จะโชว์ Error
        audio_path = cloud_tts(processed_text, emo)

        if audio_path is None:
             return None, "Error: สร้างเสียงไม่สำเร็จ (TTS Failed)", emo

        print("✅ Success!")
        return audio_path, processed_text, emo

    except Exception as e:
        print(f"🔥 CRITICAL ERROR: {e}")
        return None, f"System Error: {str(e)}", "Crash"

# --- UI Setup ---
demo = gr.Interface(
    fn=run,
    inputs=[
        gr.Textbox(label="Text Input (พิมพ์ข้อความ)"),
        gr.Audio(label="Voice Input (อัดเสียง/อัปโหลด)", type="filepath"),
        gr.Checkbox(label="Auto Emotion", value=True),
        gr.Dropdown(["news","happy","sad","angry","excited"], label="Manual Emotion", value="news")
    ],
    outputs=[
        gr.Audio(label="Generated Audio"),
        gr.Textbox(label="Result / Status"),
        gr.Textbox(label="Detected Emotion")
    ],
    title="Thai Emotion TTS",
    description="ทดสอบระบบ: แก้ไขปัญหาเสียงเงียบและ Error เรียบร้อยแล้ว",
    allow_flagging="manual",
    flagging_dir="my_flagged_data"
)

demo.launch(share=True, debug=True)